In [ ]:
import pandas as pd
import numpy as np
import forecast

In [ ]:
# Загрузим данные
candles_df = forecast.get_df('BTCUSD', '2024-01-02 00:00', '1T')
predictions = pd.read_csv('forecasts/BTCUSD_2024-01-01.csv')
predictions['last_candle_time'] = pd.to_datetime(predictions['last_candle_time'])

# Функция для расчета фактических движений цены вверх
def calculate_actual_movements(predictions, candles_df):
    actual_movements = []
    
    for idx, row in predictions.iterrows():
        last_candle_time = row['last_candle_time']
        p_up = row['p_up']
        
        # Найдем цену закрытия в момент прогноза
        close_price = candles_df.loc[last_candle_time]['close']
        
        # Найдем цену открытия через пять минут
        future_time = last_candle_time + pd.Timedelta(minutes=5)
        open_price = candles_df.loc[future_time]['open']
        
        # Сравним цены
        if open_price > close_price:
            actual_movements.append(1)
        else:
            actual_movements.append(0)
    
    return actual_movements

# Рассчитаем фактические движения
predictions['actual_up'] = calculate_actual_movements(predictions, candles_df)

In [ ]:
# Получаем уникальные значения вероятности p_up
unique_p_up_values = predictions['p_up'].unique()

# Словарь для хранения результатов
p_up_to_actual_probability = {}

# Рассчитываем фактическую вероятность для каждого уникального значения p_up
for p_up in unique_p_up_values:
    # Фильтруем строки с текущим значением p_up
    filtered_predictions = predictions[predictions['p_up'] == p_up]
    
    # Рассчитываем фактическую вероятность роста для текущего значения p_up
    actual_probability = sum(filtered_predictions['actual_up']) / len(filtered_predictions['actual_up'])
    sample_size = len(filtered_predictions['actual_up'])
    p_up_to_actual_probability[p_up] = (actual_probability, sample_size)

# Выводим результаты
for forecasted_p_up, (actual_p_up, sample_size) in sorted(p_up_to_actual_probability.items()):
    print(f'Forecasted p_up: {forecasted_p_up}%, Actual p_up: {actual_p_up * 100:.2f}%, Sample size: {sample_size}')


In [ ]:
# вычисляю какой размер выборки нужен, для того, чтобы на 95% быть уверенным, что если прогноз дает вероятность 65%, что
# цена пойдет вверх, то действительно вероятность лежит в пределах +-5%, т.е. в диапозоне от 60% до 70%.
import math

# Заданные значения
Z = 1.96 # z-score для уверенности 95%
p = 0.65 # пропорция популяции (65%)
E = 0.05 # диапозон +-5% для 65%

# Формула для размера выборки
n = (Z**2 * p * (1 - p)) / E**2
n_rounded = math.ceil(n)  # Округлим до ближайшего целого значения

n_rounded


In [ ]:
# выведу данные прогноза на график цены. диапозон, вероятность, уверенность.
%matplotlib widget
#%matplotlib inline
import matplotlib.pyplot as plt
from mpl_interactions import panhandler, zoom_factory
import mplfinance as mpf
import ipywidgets as widgets
#%matplotlib notebook

In [ ]:
# стиль графика для mpl. пытаюсь сделать что-то похожее на светлую тему tradingview.
customstyle = mpf.make_mpf_style(base_mpf_style='yahoo', facecolor='w')
dpi = 96
width_in_inches = 936 / dpi
height_in_inches = 568 / dpi

In [ ]:
first = predictions.iloc[0]['last_candle_time']
last = predictions.iloc[-1]['last_candle_time']
first = first + pd.Timedelta(minutes=-200)
last = last + pd.Timedelta(minutes=1)

In [ ]:
#fig, ax = plt.subplots()


# apdict = [
#     mpf.make_addplot(predictions['high'], color='red'),
#     mpf.make_addplot(predictions['low'], color='blue'),
#     ]

# fix, ax = mpf.plot(
#     candles_df[first:last],
#     type='candle',
#     volume=True,
#     figsize=(width_in_inches, height_in_inches),
#     addplot=apdict,
#     returnfig=True,
#     style=customstyle)
# #ax.plot(candles_df['open'][first:last], 'b-')
# #display(widgets.VBox([fix.canvas]))
# #fix.subplots_adjust(left=0, right=1, top=1, bottom=0)
# #ax[0].margins(x=0)

# # Настраиваем отступы
# fix.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

# plt.tight_layout(pad=0)

# plt.show()

In [ ]:
#!pip install plotly


In [ ]:
h = predictions['high']
h.index = df.index[-predictions.shape[0]:]
he5 = ta.sma(predictions['high'], length=3)
p = df['close'][-predictions.shape[0]:]
# Обновление индекса ema, чтобы он соответствовал индексу df
he5.index = df.index[-predictions.shape[0]:]
#h - df['close'][-predictions.shape[0]:]
l = predictions['low']
l.index = df.index[-predictions.shape[0]:]
le5 = ta.sma(predictions['low'], length=3)
le5.index = df.index[-predictions.shape[0]:]

In [ ]:
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Пример использования вашего датафрейма
df = candles_df[first:last].copy()

# Вычисляем индикаторы с помощью pandas_ta
df['SMA20'] = ta.sma(df['close'], length=20)
df['EMA20'] = ta.ema(df['close'], length=20)

# Создаем график с субплотами
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03, 
               subplot_titles=('OHLC', 'Volume / probability up', 'confidence', 'forecasted range diff current price'), 
               row_heights=[3, 1, 1, 1],
               specs=[[{"secondary_y": True}],[{"secondary_y": True}],[{"secondary_y": True}],[{"secondary_y": True}]])

# Добавляем свечной график
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['open'],
                             high=df['high'],
                             low=df['low'],
                             close=df['close'],
                             name='Candlesticks'), row=1, col=1)

# Добавляем индикаторы в первый ряд
fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=ta.ema(predictions['high'], length=5), mode='lines', name='hi e5', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=predictions['high'], mode='lines', name='hi', line=dict(color='orange')), row=1, col=1)
fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=ta.ema(predictions['low'], length=5), mode='lines', name='lo e5', line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=predictions['low'], mode='lines', name='lo', line=dict(color='lightgreen')), row=1, col=1)

fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=he5 - df['close'][-predictions.shape[0]:], mode='lines', name='hi - curr e5', line=dict(color='red')), row=4, col=1)
fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=le5 - df['close'][-predictions.shape[0]:], mode='lines', name='lo - curr e5', line=dict(color='green')), row=4, col=1)
fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=h - df['close'][-predictions.shape[0]:], mode='lines', name='hi - curr', line=dict(color='orange')), row=4, col=1)
fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=l - df['close'][-predictions.shape[0]:], mode='lines', name='lo - curr', line=dict(color='lightgreen')), row=4, col=1)

# Добавляем объемы во второй ряд
fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume'), row=2, col=1)
fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=ta.ema(predictions['conf'], length=5), name='conf e5'), row=3, col=1)
fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=ta.ema(predictions['conf'], length=3), name='conf e3'), row=3, col=1)
fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=ta.sma(predictions['conf'], length=150), name='conf e100'), row=3, col=1)
fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=predictions['p_up'], name='p_up', line=dict(color='red')), row=2, col=1, secondary_y=True)
fig.add_trace(go.Scatter(x=predictions['last_candle_time'], y=ta.ema(predictions['p_up'], length=3), name='p_up e3', line=dict(color='blue')), row=2, col=1, secondary_y=True)

# Настраиваем внешний вид графика
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
    margin=dict(
        l=20,
        r=20,
        b=20,
        t=20,
        pad=0
    ),
    xaxis=dict(
        showspikes=True,
        spikemode='across',
        spikesnap='cursor',
        spikedash='solid',
        spikethickness=1,
    ),
    xaxis2=dict(
        showspikes=True,
        spikemode='across',
        spikesnap='cursor',
        spikedash='solid',
        spikethickness=1,
    ),
    xaxis3=dict(
        showspikes=True,
        spikemode='across',
        spikesnap='cursor',
        spikedash='solid',
        spikethickness=1,
    ),
    xaxis4=dict(
        showspikes=True,
        spikemode='across',
        spikesnap='cursor',
        spikedash='solid',
        spikethickness=1,
    ),
    hovermode='x',
    dragmode='pan'  # Устанавливаем режим панорамирования по умолчанию
)

fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()
